In [ ]:
!pip install --quiet mrjob

     |████████████████████████████████| 439 kB 27.8 MB/s 


In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
%%file temp.py

class FPNode:
    def __init__(self, item, count, parent):
        self.item = item
        self.count = count
        self.parent = parent
        self.children = {}
        self.link = None

    def add(self, transaction, item_freq):
        if transaction:
            item = transaction[0]
            child = self.children.get(item)
            if child is None:
                child = FPNode(item, item_freq[item], self)
                self.children[item] = child
                if self.link is not None:
                    child.link = self.link.children.get(item)
                    self.link.children[item] = child
                else:
                    child.link = None
            child.count += 1
            child.add(transaction[1:], item_freq)

    def prefix_paths(self, item):
        paths = []
        if self.item == item:
            paths.append([self])
        for child in self.children.values():
            for path in child.prefix_paths(item):
                paths.append([self] + path)
        return paths

Writing temp.py


In [ ]:
%%file PFP.py
from temp import FPNode
from mrjob.job import MRJob
from mrjob.step import MRStep

min_support=2

class PFPGrowth(MRJob):
    def mapper(self, _, value):
        items = value.strip().split()
        for item in items:
            yield item, 1

    def reducer(self, key, value):
        count=sum(value)
        if(count>=min_support):
            yield key, count

    def mapper1(self, key, value):
        

    def reducer1(self, key, value):
        

Writing PFP.py


In [ ]:
%%file temp1.py

# class FPNode:
#     def __init__(self, value, count, parent):
#         self.value = value
#         self.count = count
#         self.parent = parent
#         self.children = []
#         self.link = None

#     def has_child(self, value):
#         for node in self.children:
#             if node.value == value:
#                 return True
#         return False

#     def get_child(self, value):
#         for node in self.children:
#             if node.value == value:
#                 return node
#         return None

#     def add_child(self, value, count=1):
#         child = FPNode(value, count, self)
#         self.children.append(child)
#         return child

class FPTree:
    def __init__(self, min_support=1):
        self.root = FPNode(None, None, None)
        self.min_support = min_support
        self.header_table = {}

    def add(self, items, count=1):
        current = self.root
        for item in items:
            child = current.get_child(item)
            if child is not None:
                child.count += count
            else:
                child = current.add_child(item, count)
                if item in self.header_table:
                    self.update_header(item, child)
                else:
                    self.header_table[item] = [child, 0]
            current = child
        current.count += count

    def update_header(self, item, node):
        previous = self.header_table[item][0]
        previous.link = node
        self.header_table[item][0] = node

    def find_frequent_patterns(self):
        patterns = {}
        for item in self.header_table:
            node = self.header_table[item][0]
            while node is not None:
                pattern = self.get_pattern(node)
                count = node.count
                if count >= self.min_support:
                    patterns[tuple(pattern)] = count
                node = node.link
        return patterns

    def get_pattern(self, node):
        pattern = []
        current = node
        while current.parent is not None:
            pattern.append(current.value)
            current = current.parent
        return pattern[::-1]

    def to_json(self):
        # Convert the tree to a JSON object
        return {
            'min_support': self.min_support,
            'root': self.root.to_json(),
            'header_table': self.header_table
        }

    @staticmethod
    def from_json(tree_json):
        # Load the tree from a JSON object
        tree = FPTree(tree_json['min_support'])
        tree.root = FPNode.from_json(tree_json['root'])
        tree.header_table = tree_json['header_table']
        return tree

class FPNode:
    def __init__(self, value, count, parent):
        self.value = value
        self.count = count
        self.parent = parent
        self.children = []
        self.link = None

    def has_child(self, value):
        for node in self.children:
            if node.value == value:
                return True
        return False

    def get_child(self, value):
        for node in self.children:
            if node.value == value:
                return node
        return None

    def add_child(self, value, count=1):
        child = FPNode(value, count, self)
        self.children.append(child)
        return child

    def to_json(self):
        # Convert the node to a JSON object
        return {
            'value': self.value,
            'count': self.count,
            'children': [node.to_json() for node in self.children]
        }

    @staticmethod
    def from_json(node_json):
        # Load the node from a JSON object
        node = FPNode(node_json['value'], node_json['count'], None)
        node.children = [FPNode.from_json(child) for child in node_json['children']]
        return node


Writing temp1.py


In [ ]:
%%file PFP1.py
from temp1 import FPTree
from temp1 import FPNode
from mrjob.job import MRJob
from mrjob.step import MRStep

class MRFPGrowth(MRJob):

    def steps(self):
        return [
            MRStep(mapper=self.mapper_get_items,
                   reducer=self.reducer_count_items),
            MRStep(mapper=self.mapper_sort_counts,
                   mapper_init=self.load_frequent_itemsets,
                   reducer=self.reducer_build_fp_tree),
            MRStep(mapper=self.mapper_find_patterns,
                   reducer=self.reducer_output_patterns)
        ]

    def mapper_get_items(self, _, line):
        # Split the input line into a list of items
        items = line.split()
        # Emit each item in the list with a count of 1
        for item in items:
            yield item, 1

    def reducer_count_items(self, item, counts):
        # Sum the counts of the items
        total_count = sum(counts)
        # Emit the item and its total count
        yield item, total_count

    def mapper_sort_counts(self, item, count):
        # Load the frequent itemsets
        frequent_itemsets = self.frequent_itemsets
        # If the item is frequent, emit it with its count
        if item in frequent_itemsets:
            yield frequent_itemsets[item], (count, item)

    def reducer_build_fp_tree(self, _, counts_and_items):
        # Build the FP-tree
        tree = FPTree()
        for count, item in counts_and_items:
            tree.add(item, count)
        # Emit the tree as a JSON object
        yield None, tree.to_json()

    def mapper_find_patterns(self, _, tree_json):
        # Load the tree from the JSON object
        tree = FPTree.from_json(tree_json)
        # Find the frequent patterns in the tree
        patterns = tree.find_frequent_patterns()
        # Emit each pattern with a count of 1
        for pattern in patterns:
            yield pattern, 1

    def reducer_output_patterns(self, pattern, counts):
        # Sum the counts of the patterns
        total_count = sum(counts)
        # Emit the pattern and its total count
        yield pattern, total_count

    def load_frequent_itemsets(self):
    # Load the frequent itemsets from a file
        with open("/content/gdrive/MyDrive/DataSetA.csv", 'r') as f:
            frequent_itemsets = {}
            for line in f:
                items = tuple(item for item in line.strip().split(','))
                # items = tuple(items.split(','))
                # count = int(count)
                frequent_itemsets[items[0]] = items[1]
        # Store the frequent itemsets in the MRJob instance
        self.frequent_itemsets = frequent_itemsets


if __name__ == '__main__':
    # MRFPGrowth.run()
    inputFile = "/content/gdrive/MyDrive/DataSetA.csv"
    outFile =  "flist.csv"
    mr_job = MRFPGrowth(args=[inputFile])
    with mr_job.make_runner() as runner:
      runner.run()
      f = open(outFile, "w") 
      for key, value in mr_job.parse_output(runner.cat_output()):
        s = f'{key},{value}'
        print(s, file = f)
      f.close()

    mr_job2 = MRFPGrowth(args=[inputFile])
    with mr_job2.make_runner() as runner:
      runner.run()
      for key, value in mr_job2.parse_output(runner.cat_output()):
        print(key,value)

Overwriting PFP1.py


In [ ]:
!python PFP1.py  '--no-bootstrap-mrjob' "/content/gdrive/MyDrive/DataSetA.csv"

No configs specified for inline runner
No configs specified for inline runner
